# Setup

In [1]:
# -*- coding: utf-8 -*-
import gensim
import os
import re
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib
import deepcut
import csv
from sklearn.cluster import KMeans
import pickle
from IPython.display import display

from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

#Never truncate example
pd.set_option('display.max_colwidth', -1)

C:\Users\Administrator\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


# Load

In [2]:
model = gensim.models.Word2Vec.load("./wikisave/wiki.model")
df = pd.read_csv("./wikisave/wikidf.csv",encoding='utf-8')
df = df[['cluster','word']]
df.set_index('word',inplace=True)

# read python dict back from the file
pkl_file = open('./wikisave/dictionary.pkl', 'rb')
dic = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./wikisave/freqdictionary.pkl', 'rb')
freqdic = pickle.load(pkl_file)
pkl_file.close()

# Main functions

In [3]:
def findme(freqdic,dic,x,showbadclusters=True,n=5):
    rv = {}
    try:
        clusterdata = dic[x]
        for k in clusterdata:
            rv[k] = freqdic.get(k,0)
        #Descending order of frequency
        if len(rv.keys())<n:
            return 'Warning: This cluster has less than ' + str(n) + ' elements. ' + str(list(rv.keys())[:n])
        return list(rv.keys())[:n]
    except:
        print(x,' is in too large of a cluster. Ommitting the cluster.')
        return np.nan

def meaning(freqdic,model,line,dfformodel,dic,showbadclusters=True,n=5,max=10):
    w1 = deepcut.tokenize(line)    
    w2 = []
    for word in w1:
        if word not in model.wv.vocab:
            print(word + " not part of model's vocabulary. REMOVING")
            w1.remove(word)
    if w1 == []:
        print('All words in the sentence are not in the model. Sorry, score cannot be computed.')
        return None
    df = pd.DataFrame()
    wordscore = df.from_records(model.wv.most_similar(positive=w1,negative=w2,topn=25),columns=['word','score'])\
    .set_index('word')
    wordscore = wordscore[wordscore['score']>=wordscore['score'].max()-0.3]
    main = wordscore.join(dfformodel, how='inner')
    main['example'] = ''
    main[['example']] = main['cluster'].apply(lambda x: findme(freqdic,dic,x,showbadclusters,n))
    main.dropna(inplace=True,how='any')
    essence(main)
    return main[:max]

def essence(df):
    essence = df[:100]
    grouped = essence.groupby('cluster')['score'].sum().sort_values(ascending=False).to_frame().reset_index()
    grouped['clusterelements'] =''
    grouped[['clusterelements']] = grouped['cluster'].apply(lambda x: dic[x])
    grouped.set_index('cluster',inplace=True)
    print('\033[1m' + 'Top 3 clusters ranked by sum of score from top 100 words similar to input' + '\033[0m')
    display(grouped[:3])


# Test Word2Vec

In [18]:
w1 = ['ราชา','หญิง']
w2 = ['ชาย']
print('# of unique words in model: ' + str(len(model.wv.vocab.keys())))
model.wv.most_similar(positive=w1,negative=w2,topn=8)

# of unique words in model: 9538


[('ราชินี', 0.5389056205749512),
 ('ราชินีนาถ', 0.5355395078659058),
 ('สังฆราช', 0.4509378671646118),
 ('อัครมเหสี', 0.44618910551071167),
 ('น้อง', 0.4299611449241638),
 ('ราชาธิราช', 0.42396727204322815),
 ('สวรินทิรา', 0.4162847697734833),
 ('ราเมศวร', 0.41443413496017456)]

In [19]:
model.wv.most_similar(positive=['เงิน'],topn=8)

[('ทุนทรัพย์', 0.5699750185012817),
 ('สินทรัพย์', 0.5402774214744568),
 ('หนี้สิน', 0.5253497958183289),
 ('ดอกเบี้ย', 0.5246715545654297),
 ('พันธบัตร', 0.5164479613304138),
 ('หนี้', 0.5133445858955383),
 ('งบประมาณ', 0.5064982175827026),
 ('ธนบัตร', 0.501731276512146)]

# Test Cluster

In [19]:
df[df['cluster']==50][:10]

,cluster
word,
ประเทศ,50
ชาติ,50
พรรค,50
รัฐ,50
รัฐบาล,50
สภา,50
สหรัฐ,50
สาธารณรัฐ,50
สมาคม,50


# Test Word2Vec + Cluster (Single Word)

In [20]:
string = 'บิน'
meaning(freqdic,model,string,df,dic,showbadclusters=True,n=10,max=10)

Top 3 clusters ranked by sum of score from top 100 words similar to input


,score,clusterelements
cluster,,
115,2.855745,"[ทาง, สาย, สถานี, ถนน, ท่า, เดิน, รถไฟ, เส้นทาง, สะพาน, ขนส่ง, ทางหลวง, วิ่ง, ขบวน, รถไฟฟ้า, กิโลเมตร, ด่าน, กั้น, คมนาคม, ราง, โดยสาร, เมตร, สนามบิน, จราจร, จักรยาน, อ้อม, ปลายทาง, อุโมงค์, เลี้ยว, สี่แยก, แล่น, ล่อง, ฟาก, โทรเลข, ทางออก, ด่วน, สัญจร, เลียบ, ชานชาลา, ไมล์, ลอด, วงเวียน, เกวียน, คบเพลิง, ทางด่วน, รถราง, แท็กซี่, ทางผ่าน, ขับขี่, ดาดฟ้า, รถม้า, รันเวย์, ลาดยาง, รถเมล์, เรือโดยสาร]"
197,2.164459,"[เรือ, รถ, บิน, อวกาศ, เครื่องบิน, อากาศยาน, ยาน, รถยนต์, เครื่องยนต์, ผู้โดยสาร, จอด, นักบิน, ยานอวกาศ, เฮลิคอปเตอร์, ยานพาหนะ, กระสวย, รถบรรทุก]"
186,1.950669,"[กำลัง, อากาศ, ฐาน, อาวุธ, บก, ปฏิบัติการ, ปืน, ถัง, ติดตั้ง, ประจำการ, กระสุน, ปืนใหญ่, สูท, บรรทุก, จรวด, ซ่อม, เป้า, เรือรบ, ขีปนาวุธ, ลูกเรือ, กองพล, เสบียง, พิสัย, พาหนะ, กองพัน, สมรรถนะ, แนวหน้า, ยุทธวิธี, ดีเซล, ราชนาวี, ยนต์, ปืนกล, ประจัญบาน, เรือดำน้ำ, แนวรบ, นาวิกโยธิน, เรือหลวง, ไรเฟิล, ลาดตระเวน, รถจักร, ปืนเล็กยาว, ลำกล้อง, ใบพัด, ระวาง, กระสุนปืน, นาวี, กราด, ยุทโธปกรณ์, ยานเกราะ, ดินปืน, พลร่ม, สอดแนม, ทุ่น, กะลาสี, กองหนุน, คาบศิลา, กองร้อย, ตอร์ปิโด, ปืนพก, สัมภาระ, เรือพิฆาต, อิเลคทรอนิก, ยุทธภัณฑ์]"


,score,cluster,example
word,,,
แอร์,0.518370,83,"[บี, ที, ไอ, เอส, เดอะ, พี, เอ็ม, นิว, ซูเปอร์, ชัน]"
ขนส่ง,0.498185,115,"[ทาง, สาย, สถานี, ถนน, ท่า, เดิน, รถไฟ, เส้นทาง, สะพาน, ขนส่ง]"
เครื่องบิน,0.464524,197,"[เรือ, รถ, บิน, อวกาศ, เครื่องบิน, อากาศยาน, ยาน, รถยนต์, เครื่องยนต์, ผู้โดยสาร]"
นักบิน,0.459554,197,"[เรือ, รถ, บิน, อวกาศ, เครื่องบิน, อากาศยาน, ยาน, รถยนต์, เครื่องยนต์, ผู้โดยสาร]"
อากาศยาน,0.447323,197,"[เรือ, รถ, บิน, อวกาศ, เครื่องบิน, อากาศยาน, ยาน, รถยนต์, เครื่องยนต์, ผู้โดยสาร]"
รถไฟ,0.440395,115,"[ทาง, สาย, สถานี, ถนน, ท่า, เดิน, รถไฟ, เส้นทาง, สะพาน, ขนส่ง]"
สนามบิน,0.439632,115,"[ทาง, สาย, สถานี, ถนน, ท่า, เดิน, รถไฟ, เส้นทาง, สะพาน, ขนส่ง]"
ไลน์,0.438975,83,"[บี, ที, ไอ, เอส, เดอะ, พี, เอ็ม, นิว, ซูเปอร์, ชัน]"
ลาดตระเวน,0.436676,186,"[กำลัง, อากาศ, ฐาน, อาวุธ, บก, ปฏิบัติการ, ปืน, ถัง, ติดตั้ง, ประจำการ]"


# Test Word2Vec + Cluster (Single Sentence)

In [21]:
string = 'ขอบคุณทุกหน่วยงานที่ให้ความช่วยเหลือเด็กๆ'
meaning(freqdic,model,string,df,dic,showbadclusters=True,n=10,max=10)

Top 3 clusters ranked by sum of score from top 100 words similar to input


,score,clusterelements
cluster,,
108,1.824959,"[เพื่อ, ประชาชน, จัดการ, โครงการ, ท้องถิ่น, ประโยชน์, เทศบาล, ศูนย์, ชุมชน, กิจกรรม, ส่งเสริม, เกษตร, สาธารณะ, เอกชน, ทรัพยากร, แรงงาน, บำรุง, มูลนิธิ, อนุรักษ์, เพาะ, เกษตรกรรม, ชนบท, เหมือง, บุคลากร, ประมง, ส่วนกลาง, ประชาสัมพันธ์, นิคม, กองทุน, สหกรณ์, อนามัย, ยูเนสโก, บรรเทา, เกษตรกร, สุขาภิบาล, ประหยัด, หมุนเวียน, มหาชน, ชลประทาน, ฟาร์ม, โทรคมนาคม, เสริมสร้าง, สวัสดิการ, ผลิตผล, สงเคราะห์, ใช้สอย, พอเพียง, นิติบุคคล, ปศุ, สาธารณูปโภค, บูรณาการ, ศุลกากร, สัมมนา, ประปา, สร้างเสริม, อุปโภค, กสิกรรม, สาธารณประโยชน์, วิสาหกิจ, เร่งรัด, น้ำประปา, นันทนาการ]"
44,1.205040,"[ปัญหา, สังคม, เศรษฐกิจ, นโยบาย, ให้การ, สุขภาพ, สิ่งแวดล้อม, ว่าด้วย, สาธารณสุข, การเงิน, มวลชน, เสรีภาพ, รณรงค์, ยุทธศาสตร์, มนุษย, อาชญากรรม, ยาเสพติด, จริยธรรม, ศีลธรรม, ฉ้อ, มนุษยธรรม]"
84,0.773332,"[บริษัท, ขาย, บริการ, จำหน่าย, ค้า, ธุรกิจ, ตลาด, อุตสาหกรรม, สินค้า, กิจการ, ร้าน, ธนาคาร, ผลิตภัณฑ์, โรงงาน, พาณิชย์, โรงแรม, หลักทรัพย์, ลูกค้า, ห้าง, พ่อค้า, ศูนย์การค้า, ผู้บริโภค, ประมูล, ยี่ห้อ, แบรนด์, อสังหาริมทรัพย์, รัฐวิสาหกิจ, ของเล่น, ผู้ประกอบการ, หุ้นส่วน, โตโยต้า, ปลีก, เครื่องสำอาง, ฮอนด้า, ประกันภัย, ภัตตาคาร, แฟรนไชส์, นิสสัน, บรรษัท, โภคภัณฑ์]"


,score,cluster,example
word,,,
ให้การ,0.485948,44,"[ปัญหา, สังคม, เศรษฐกิจ, นโยบาย, ให้การ, สุขภาพ, สิ่งแวดล้อม, ว่าด้วย, สาธารณสุข, การเงิน]"
ทุกๆ,0.434746,1,"[นี้, นั้น, หนึ่ง, ตาม, แรก, ใหม่, หลาย, บาง, ประกอบ, อื่น]"
สวัสดิภาพ,0.402155,88,"[ที่, ใน, เป็น, และ, ได้, โดย, ซึ่ง, กัน, ยัง, ทั้ง]"
ลูกค้า,0.397653,84,"[บริษัท, ขาย, บริการ, จำหน่าย, ค้า, ธุรกิจ, ตลาด, อุตสาหกรรม, สินค้า, กิจการ]"
เด็กๆ,0.394260,25,"[หญิง, อายุ, เด็ก, เพศ, วัย, สตรี, หนุ่ม, ผู้หญิง, ธรรมดา, ผู้ใหญ่]"
สวัสดิการ,0.388642,108,"[เพื่อ, ประชาชน, จัดการ, โครงการ, ท้องถิ่น, ประโยชน์, เทศบาล, ศูนย์, ชุมชน, กิจกรรม]"
หมั่น,0.384622,64,"[จึง, จน, กลับ, ขณะ, เอง, พา, ตัวเอง, นี่, เสมอ, นั่น]"
ลูกจ้าง,0.377173,28,"[ตำแหน่ง, คณะ, นาย, กรม, ราชการ, หน่วย, กระทรวง, หน้าที่, นายก, สังกัด]"
ผู้บริโภค,0.375679,84,"[บริษัท, ขาย, บริการ, จำหน่าย, ค้า, ธุรกิจ, ตลาด, อุตสาหกรรม, สินค้า, กิจการ]"


# Test Word2Vec + Cluster (Multi Sentence: Comment from SCB Thailand)

In [22]:
string = 'ขอบคุณทุกหน่วยงานที่ให้ความช่วยเหลือเด็กๆ ฉันเองก็คงจะหลับสบายๆไม่ตื่นสะอื้นกลางดึกเพราะห่วงคนแปลกหน้า'.split()
for i in string:
    display(meaning(freqdic,model,i,df,dic,showbadclusters=True,n=10,max=10))

Top 3 clusters ranked by sum of score from top 100 words similar to input


,score,clusterelements
cluster,,
108,1.824959,"[เพื่อ, ประชาชน, จัดการ, โครงการ, ท้องถิ่น, ประโยชน์, เทศบาล, ศูนย์, ชุมชน, กิจกรรม, ส่งเสริม, เกษตร, สาธารณะ, เอกชน, ทรัพยากร, แรงงาน, บำรุง, มูลนิธิ, อนุรักษ์, เพาะ, เกษตรกรรม, ชนบท, เหมือง, บุคลากร, ประมง, ส่วนกลาง, ประชาสัมพันธ์, นิคม, กองทุน, สหกรณ์, อนามัย, ยูเนสโก, บรรเทา, เกษตรกร, สุขาภิบาล, ประหยัด, หมุนเวียน, มหาชน, ชลประทาน, ฟาร์ม, โทรคมนาคม, เสริมสร้าง, สวัสดิการ, ผลิตผล, สงเคราะห์, ใช้สอย, พอเพียง, นิติบุคคล, ปศุ, สาธารณูปโภค, บูรณาการ, ศุลกากร, สัมมนา, ประปา, สร้างเสริม, อุปโภค, กสิกรรม, สาธารณประโยชน์, วิสาหกิจ, เร่งรัด, น้ำประปา, นันทนาการ]"
44,1.205040,"[ปัญหา, สังคม, เศรษฐกิจ, นโยบาย, ให้การ, สุขภาพ, สิ่งแวดล้อม, ว่าด้วย, สาธารณสุข, การเงิน, มวลชน, เสรีภาพ, รณรงค์, ยุทธศาสตร์, มนุษย, อาชญากรรม, ยาเสพติด, จริยธรรม, ศีลธรรม, ฉ้อ, มนุษยธรรม]"
84,0.773332,"[บริษัท, ขาย, บริการ, จำหน่าย, ค้า, ธุรกิจ, ตลาด, อุตสาหกรรม, สินค้า, กิจการ, ร้าน, ธนาคาร, ผลิตภัณฑ์, โรงงาน, พาณิชย์, โรงแรม, หลักทรัพย์, ลูกค้า, ห้าง, พ่อค้า, ศูนย์การค้า, ผู้บริโภค, ประมูล, ยี่ห้อ, แบรนด์, อสังหาริมทรัพย์, รัฐวิสาหกิจ, ของเล่น, ผู้ประกอบการ, หุ้นส่วน, โตโยต้า, ปลีก, เครื่องสำอาง, ฮอนด้า, ประกันภัย, ภัตตาคาร, แฟรนไชส์, นิสสัน, บรรษัท, โภคภัณฑ์]"


,score,cluster,example
word,,,
ให้การ,0.485948,44,"[ปัญหา, สังคม, เศรษฐกิจ, นโยบาย, ให้การ, สุขภาพ, สิ่งแวดล้อม, ว่าด้วย, สาธารณสุข, การเงิน]"
ทุกๆ,0.434746,1,"[นี้, นั้น, หนึ่ง, ตาม, แรก, ใหม่, หลาย, บาง, ประกอบ, อื่น]"
สวัสดิภาพ,0.402155,88,"[ที่, ใน, เป็น, และ, ได้, โดย, ซึ่ง, กัน, ยัง, ทั้ง]"
ลูกค้า,0.397653,84,"[บริษัท, ขาย, บริการ, จำหน่าย, ค้า, ธุรกิจ, ตลาด, อุตสาหกรรม, สินค้า, กิจการ]"
เด็กๆ,0.394260,25,"[หญิง, อายุ, เด็ก, เพศ, วัย, สตรี, หนุ่ม, ผู้หญิง, ธรรมดา, ผู้ใหญ่]"
สวัสดิการ,0.388642,108,"[เพื่อ, ประชาชน, จัดการ, โครงการ, ท้องถิ่น, ประโยชน์, เทศบาล, ศูนย์, ชุมชน, กิจกรรม]"
หมั่น,0.384622,64,"[จึง, จน, กลับ, ขณะ, เอง, พา, ตัวเอง, นี่, เสมอ, นั่น]"
ลูกจ้าง,0.377173,28,"[ตำแหน่ง, คณะ, นาย, กรม, ราชการ, หน่วย, กระทรวง, หน้าที่, นายก, สังกัด]"
ผู้บริโภค,0.375679,84,"[บริษัท, ขาย, บริการ, จำหน่าย, ค้า, ธุรกิจ, ตลาด, อุตสาหกรรม, สินค้า, กิจการ]"


สะอื้น not part of model's vocabulary. REMOVING
Top 3 clusters ranked by sum of score from top 100 words similar to input


,score,clusterelements
cluster,,
64,5.612735,"[จึง, จน, กลับ, ขณะ, เอง, พา, ตัวเอง, นี่, เสมอ, นั่น, หัน, ทันที, ผู้คน, คอย, หลง, ทัน, สัก, ยาม, แอบ, หมอ, ชวน, ครั้น, ใส, หลับ, รีบ, ท่ามกลาง, ตื่น, ลุก, ขัน, เค้า, ดั่ง, อย่า, นิ่ง, สาวๆ, พอดี, สบาย, ลืม, เมา, ว่าที่, หลวงปู่, ลำพัง, เงียบ, แถม, พลอย, บางที, มากๆ, ยิ้ม, นวล, ปาน, ชิต, หวน, ค้าง, แสบ, อาย, น้ำตา, เพื่อนๆ, รุ่งขึ้น, ร้องไห้, แกล้ง, เที่ยง, ซากุระ, รุ้ง, บ่อยๆ, จวน, ถ้วน, บัดนี้, ก้อง, เจ้าตัว, มาบ, ท่านหญิง, อ้าย, หัวเราะ, เยอะ, กานต์, ทันใด, เหนื่อย, ตะโกน, มัว, โอ้, หลอน, หล่อน, อิ่ม, แวว, ขนุน, ทับทิม, ลุ้น, ตั้งท้อง, ดึก, สึก, พูดจา, อึ้ง, เถิด, สะดุด, พิม, ช่า, เอี้ยก้วย, ทั้งนั้น, เสาะ, ครับ, ล้า, ...]"
98,5.520681,"[จะ, ว่า, ไม่, ก็, แต่, เมื่อ, หรือ, แล้ว, ต้อง, อาจ, ก่อน, เพียง, เพราะ, เคย, มัก, เรา, คง, น่า, ใด, เท่านั้น, ถ้า, หาก, พอ, เลย, เกิน, เท่า, ใช่, ควร, ใคร, บ้าง, อะไร, ค่อย, แค่, อยาก, ซ้ำ, ฉัน, แทบ, ไหน, ทว่า, ใดๆ, ดูเหมือน, แต่ว่า, แน่ชัด, ฉะนั้น, เว้น, สมควร, ข้าพเจ้า, เจาะจง, จริงๆ, แน่, กระนั้น, เท่าใด, เมื่อใด, รู้ตัว, เท่าไร, ไหร่, ที่ไหน, ยุคมืด, ยังไง, หรอก, ข้อแม้, เมื่อไร, ทีท่า]"
53,1.068281,"[ใจ, จริง, รัก, พยายาม, ผิด, ชอบ, สนใจ, รู้สึก, จำ, หวัง, สุข, ฝัน, กล้า, กลัว, ตั้งใจ, พอใจ, เคารพ, แค้น, พระทัย, เปรียบ, ศรัทธา, บริสุทธิ์, ทุกข์, ปรารถนา, กังวล, ประทับใจ, โกรธ, จดจำ, ชื่นชม, เชื่อถือ, ฉลาด, ยากจน, ยินดี, หาญ, พิการ, เสียใจ, เกลียด, มั่นใจ, สามัคคี, สนุก, ผูกพัน, ตกใจ, ท้าทาย, จงรัก, สนุกสนาน, ถนัด, ชัง, เบื่อ, มุ่งมั่น, เลื่อมใส, ร่ำรวย, รู้เรื่อง, เดือดร้อน, ใส่ใจ, ผิดหวัง, เอาใจ, กระตือรือร้น, ชอบธรรม, เศร้า, อดทน, ตื่นเต้น, ดุร้าย, ซื่อสัตย์, ภูมิใจ, ดีใจ, หวาดกลัว, ทุ่มเท, ภาคภูมิ, เคือง, โหดร้าย, เป็นห่วง, หลงใหล, พึงพอใจ, เสียสละ, อัศจรรย์, ก้าวร้าว, โปรดปราน, ประมาท, สงสาร, ใฝ่ฝัน, ระแวง, เสียดาย, รักใคร่, อ่อนโยน, ตะลึง, ว่องไว, กตัญญู, รังเกียจ, อิจฉา, เก่งกาจ, ไว้ใจ, ทะเยอทะยาน, ชอบใจ, หิว, ร้ายกาจ, สมัครใจ, ขยัน, เหงา, เกรงกลัว, รอบรู้, ...]"


,score,cluster,example
word,,,
เหงา,0.586100,53,"[ใจ, จริง, รัก, พยายาม, ผิด, ชอบ, สนใจ, รู้สึก, จำ, หวัง]"
ร้องไห้,0.528408,64,"[จึง, จน, กลับ, ขณะ, เอง, พา, ตัวเอง, นี่, เสมอ, นั่น]"
ถ้า,0.527997,98,"[จะ, ว่า, ไม่, ก็, แต่, เมื่อ, หรือ, แล้ว, ต้อง, อาจ]"
อะไร,0.527666,98,"[จะ, ว่า, ไม่, ก็, แต่, เมื่อ, หรือ, แล้ว, ต้อง, อาจ]"
เผลอ,0.524528,64,"[จึง, จน, กลับ, ขณะ, เอง, พา, ตัวเอง, นี่, เสมอ, นั่น]"
สบายใจ,0.523509,64,"[จึง, จน, กลับ, ขณะ, เอง, พา, ตัวเอง, นี่, เสมอ, นั่น]"
เฉย,0.519486,64,"[จึง, จน, กลับ, ขณะ, เอง, พา, ตัวเอง, นี่, เสมอ, นั่น]"
จริงๆ,0.516625,98,"[จะ, ว่า, ไม่, ก็, แต่, เมื่อ, หรือ, แล้ว, ต้อง, อาจ]"
กะรัต,0.515422,64,"[จึง, จน, กลับ, ขณะ, เอง, พา, ตัวเอง, นี่, เสมอ, นั่น]"


# Multiple sentence. Aborting K-Means approach

In [23]:
def genscorematrix(bow,maxscore):
    rv = pd.DataFrame(columns=['word','score'])
    df = pd.DataFrame()
    for word in bow:
        wordscore = df.from_records(model.wv.most_similar(positive=word,topn=10),columns=['word','score'])
        rv = rv.append(wordscore,ignore_index=True).append(pd.DataFrame([[word,1]], columns=['word','score']),ignore_index=True)
    rv.sort_values('score',inplace=True,ascending=False)
    rv.set_index('word',inplace=True)
    rv['score'] = rv['score'].apply(lambda x: x*maxscore)
    return rv

def score(comment,scorematrix,name='',verbose=True):
    l = deepcut.tokenize(comment)
    tot = 0
    for word in l:
        if word in scorematrix.index:
            tot += scorematrix.loc[word]['score'].max()
    #print(comment)
    if verbose == True:
        print(name,': ',tot)
    return tot

# Main Functions

In [24]:
def clean(tweet):
    if 'retweeted_status' in dir(tweet):
        try:
            return tweet.retweeted_status.full_text
        except:
            return tweet.retweeted_status.text
    else:
        try:
            return tweet.full_text
        except:
            return tweet.text

def printresult(score1,score2,scoreagg,comment):
        print(comment)
        score(comment,score1,name='Score1 (ยืม/กู้)')
        score(comment,score2,name='Score2 (ลงทุน)')
        score(comment,scoreagg,name='Aggregate Score')
        print()
        
def printsummarydf(score1,score2,scoreagg,tweets,istweet=True):
    if istweet == True:
        comments = []
        dt = []
        for tweet in tweets:
            comments.append(clean(tweet))
            dt.append(tweet.created_at.date())
        summary =pd.DataFrame({'Comment': comments,'DateTime': dt})
    else:
        comments = tweets 
        summary = pd.DataFrame(comments,columns=['Comment'])
    summary['Score1 (ยืม/กู้)'] = summary['Comment'].apply(lambda x: score(x,score1,verbose=False))
    summary['Score2 (ลงทุน)'] = summary['Comment'].apply(lambda x: score(x,score2,verbose=False))
    summary['General Score'] = summary['Comment'].apply(lambda x: score(x,scoreagg,verbose=False))
    return summary

In [25]:
bow1= {'เงิน','ยืม','กู้','แบงค์','ฝาก','ผ่อน','ถอน','ฝาก'}
bow2 = {'หุ้น','กองทุน','ดอกเบี้ย','ลงทุน','พันธบัตร'}
generalterms = {'ธนาคาร','ออม','สมัคร','สนใจ','พาณิชย์','สอบถาม'}
aggregate = bow1.union(bow2,generalterms)

score1 = genscorematrix(bow1,1)
score2 = genscorematrix(bow2,1)
scoreagg = genscorematrix(aggregate,0.66)

In [26]:
comments = ["สอบถามหน่อยคะถ้าเราจะเปิดบัตรที่สามารถผ่อนของได้ต้องทำอย่างไรคะ",\
            "สนใจกู้ซื้อบ้านใหม่ กับSCB ค้าขายไม่มีทะเบียนพานิช มีโฉนดที่ดินปลอดภาระ รายได้เดือนละ40000-50000",\
           "ไทยพาณิชย์เจ๋งมากเลยนะคะ ตะกี้โทรไปอายัดบัตรเอทีเอ็ม เจ้าหน้าที่บริการต่อสาย เพื่ออายัดธนาคารอื่นให้ด้วย ขอบคุณมาก ๆ เลยคะ รู้สึกถึงการช่วยเหลือที่สุดๆ เท่าที่จะช่วยได้",\
           "ไม่ได้กู้เงินอะไรกับ SCB ทำไมมี sms มาทวง โทรถามเจ้าหน้าที่บอกระบบส่งมาจริง ไม่ทราบว่ามีการสวมประวัติอะไรหรือไม่",\
           "อุรุกวัย ปู๊นๆๆๆๆๆๆๆๆๆ",\
           "ทำแล้ว ทำไม่ได้ พนง.บอกเครื่อค่าย(MY CAT)ไม่ร่วม เศร้าแพร็บ",\
           "สอบถามนะคะ การขอเงินคืนเนื่องจากชำระยอดบัตรเครดิตเกิน ใช้สำเนาบัตร ปปช หน้าบุกแบงค์ แล้วยังจำเป็นต้องใช้สเตจเม้นย้อนหลัง 3 เดือนด้วยหรอคะ"]

scoredf = printsummarydf(score1,score2,scoreagg,comments,istweet=False).sort_values('General Score',ascending=False)
scoredf

,Comment,Score1 (ยืม/กู้),Score2 (ลงทุน),General Score
3,ไม่ได้กู้เงินอะไรกับ SCB ทำไมมี sms มาทวง โทรถามเจ้าหน้าที่บอกระบบส่งมาจริง ไม่ทราบว่ามีการสวมประวัติอะไรหรือไม่,2.500806,0.524672,2.344371
1,สนใจกู้ซื้อบ้านใหม่ กับSCB ค้าขายไม่มีทะเบียนพานิช มีโฉนดที่ดินปลอดภาระ รายได้เดือนละ40000-50000,1.488617,0.000000,1.930328
0,สอบถามหน่อยคะถ้าเราจะเปิดบัตรที่สามารถผ่อนของได้ต้องทำอย่างไรคะ,1.000000,0.000000,1.320000
6,สอบถามนะคะ การขอเงินคืนเนื่องจากชำระยอดบัตรเครดิตเกิน ใช้สำเนาบัตร ปปช หน้าบุกแบงค์ แล้วยังจำเป็นต้องใช้สเตจเม้นย้อนหลัง 3 เดือนด้วยหรอคะ,1.000000,0.524672,1.320000
2,ไทยพาณิชย์เจ๋งมากเลยนะคะ ตะกี้โทรไปอายัดบัตรเอทีเอ็ม เจ้าหน้าที่บริการต่อสาย เพื่ออายัดธนาคารอื่นให้ด้วย ขอบคุณมาก ๆ เลยคะ รู้สึกถึงการช่วยเหลือที่สุดๆ เท่าที่จะช่วยได้,0.377064,0.523231,0.908862
5,ทำแล้ว ทำไม่ได้ พนง.บอกเครื่อค่าย(MY CAT)ไม่ร่วม เศร้าแพร็บ,0.000000,0.000000,0.327853
4,อุรุกวัย ปู๊นๆๆๆๆๆๆๆๆๆ,0.000000,0.000000,0.000000


# Twitter Score Ranking

In [32]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = None
consumer_secret = None
access_token = None
access_secret = None
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

def twitter(query="(@scb_thailand)",n=30):
    dirtytweets = list(tweepy.Cursor(api.search, q=query,result_type='recent', tweet_mode='extended').items(n))
    return printsummarydf(score1,score2,scoreagg,dirtytweets).sort_values('General Score',ascending=False)

twitterdf = twitter()
twitterdf

,Comment,DateTime,Score1 (ยืม/กู้),Score2 (ลงทุน),General Score
28,@scb_thailand ตอนนี้มีปัญหา คือใช้งานแอพ up2me และโอนเงินให้เพื่อนผ่านหมายเลขโทรศัพท์ ที่ผูกกับบัญชีกสิกร แต่ผลคือเพื่อนได้รับโอน แต่พบว่าเพื่อนผูกพร้อมเพยกับ scb ด้วยบัตรประชาชน แต่ทำไม เมื่อโอนด้วยเบอร์โทรศัพท์ ทำไมไม่เข้าบัญชีกสิกร แต่ไปเข้า scb ที่เพื่อนผูกไว้ด้วยบัตรประชาชน คับ,2018-07-17,3.451455,0.524672,2.277960
5,"สิทธิพิเศษสำหรับลูกค้า SCB\nรับส่วนลดสูงสุด 50,000 บาท*\nเมื่อจองโครงการ ภัสสร เพรสทีจ จตุโชติ-วัชรพล\nวันที่ 19-31 ก.ค. 2561 ที่ Sales Gallery\nลงทะเบียนรับส่วนลดเงินสดเพิ่ม 10,000 บาท\nที่ https://t.co/WJ14FH5zv2 \nเริ่ม 4.35 ลบ.* / โทร.1739 https://t.co/liyyh5mohv",2018-07-19,1.842088,1.095304,1.592395
6,@scb_thailand มีคนคนโอนเงินมาตั้งแต่วันที่ 17 แต่ตอนนี้เงินยังไม่เข้า พร้อมเพย์ของSCB เลยค่ะ ไม่ทราบว่าระบบมีปัญหาอะไรรึเปล่าคะ,2018-07-19,2.394174,1.049343,1.580155
22,@scb_thailand ตรวจสอบแล้วคับว่าเบอร์โทรผูกกับธนาคารกสิกรไทย แต่ถ้าหมายเลขบัตรประชาชนผูกกับไทยพานิชย์,2018-07-17,0.845955,0.000000,0.938108
13,@scb_thailand โอนพร้อมเพย์แล้วเงินไม่เข้าค่ะ 2 รอบแล้วนะ,2018-07-18,1.394174,0.524672,0.920155
18,@scb_thailand สามารถใช้งานได้ตามปกติ หลังจากเกิดปัญหาแล้วประมาณ 30 นาที ครับ ขอบคุณทางธนาคารที่ตอบกลับครับผม,2018-07-18,0.000000,0.523231,0.660000
2,#quoteoftheday\n#scbsme\n#scbbusinesscenter\nติดตามกิจกรรมที่น่าสนใจได้ที่ https://t.co/agHRwLHaxw https://t.co/e3GtMMu76N,2018-07-19,0.000000,0.000000,0.660000
3,#quoteoftheday\n#scbsme\n#scbbusinesscenter\nติดตามกิจกรรมที่น่าสนใจได้ที่ https://t.co/agHRwLHaxw https://t.co/AjnyiP2jX6,2018-07-19,0.000000,0.000000,0.660000
4,@scb_thailand อยากทราบสาขาไทยพาณิชย์ สาขาหมายเลข 8232 ไม่ทราบว่าเป็นสาขาอะไรหรอคะ,2018-07-19,0.000000,0.000000,0.660000
8,@scb_thailand เคยทำแบบนี้แล้วต้องไปแบงค์เพื่อใช้แอพใหม่ แต่ลองดูอีกทีก็ได้,2018-07-18,1.000000,0.000000,0.660000


In [174]:
datetimedf = printsummarydf(score1,score2,scoreagg,tweets)

In [184]:
display(datetimedf.sort_values('General Score',ascending=False).groupby('DateTime')["Score1 (ยืม/กู้)","Score2 (ลงทุน)"].mean())
datetimedf.sort_values('General Score',ascending=False).groupby('DateTime')["Score1 (ยืม/กู้)","Score2 (ลงทุน)"].count()

,Score1 (ยืม/กู้),Score2 (ลงทุน)
DateTime,,
2018-07-15,0.123587,0.034194
2018-07-16,0.306154,0.245061
2018-07-17,0.186047,0.310180


,Score1 (ยืม/กู้),Score2 (ลงทุน)
DateTime,,
2018-07-15,47,47
2018-07-16,108,108
2018-07-17,43,43


# TSNE Plot for visualization (Incomplete)

In [237]:
def tsne(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)
    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
    
    return x,y

def plotpic(model,savename,color,mode=False,x=None,y=None):
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)

    if mode == False:
        x,y = tsne(model)
    plt.figure(figsize=(16, 16))
    colorbar = get_cmap(30)
    
    for i,word in enumerate(labels):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.savefig(savename +'.svg', format='svg', dpi=1200)